In [1]:
%load_ext sql
# %config SqlMagic.autocommit=False # for engines that do not support autommit

In [2]:
%%sql sqlite:///Northwind2016.sqlite

select name from sqlite_master where type = 'table'

Done.


name
dbo.Categories
dbo.Suppliers
dbo.Orders
dbo.OrderDetails
dbo.Shippers
dbo.Products
dbo.CustomerGroupThresholds
dbo.Employees
dbo.sysdiagrams
dbo.Customers


## 20. Categories, and the total products in
each category
For this problem, we’d like to see the total number of
products in each category. Sort the results by the total
number of products, in descending order.

In [23]:
%%sql sqlite:///Northwind2016.sqlite

SELECT cat.CategoryName, COUNT(prod.ProductID) as count
FROM 'dbo.Categories' as cat
join 'dbo.Products' as prod
on prod.CategoryID = cat.CategoryID
GROUP BY 1
ORDER BY 2 DESC

Done.


CategoryName,count
Confections,13
Seafood,12
Condiments,12
Beverages,12
Dairy Products,10
Grains/Cereals,7
Meat/Poultry,6
Produce,5


## 21. Total customers per country/city
In the Customers table, show the total number of
customers per Country and City

In [27]:
%%sql sqlite:///Northwind2016.sqlite

SELECT City, Country, COUNT(*) as 'number_of_customers'
FROM 'dbo.Customers'
GROUP BY City, Country
ORDER BY 3 DESC
LIMIT 10

Done.


City,Country,number_of_customers
London,UK,6
México D.F.,Mexico,5
Sao Paulo,Brazil,4
Buenos Aires,Argentina,3
Madrid,Spain,3
Rio de Janeiro,Brazil,3
Lisboa,Portugal,2
Nantes,France,2
Paris,France,2
Portland,USA,2


## 22. Products that need reordering
What products do we have in our inventory that
should be reordered? For now, just use the fields
UnitsInStock and ReorderLevel, where UnitsInStock
is less than the ReorderLevel, ignoring the fields
UnitsOnOrder and Discontinued.
Order the results by ProductID.

In [46]:
%%sql sqlite:///Northwind2016.sqlite

SELECT CAST(ProductID as int), ProductName, UnitsInStock, ReorderLevel
FROM 'dbo.Products'
WHERE CAST(UnitsInStock as int) < CAST(ReorderLevel as int)
ORDER BY 1 ASC

Done.


CAST(ProductID as int),ProductName,UnitsInStock,ReorderLevel
2,Chang,17,25
3,Aniseed Syrup,13,25
11,Queso Cabrales,22,30
21,Sir Rodney's Scones,3,5
30,Nord-Ost Matjeshering,10,15
31,Gorgonzola Telino,0,20
32,Mascarpone Fabioli,9,25
37,Gravad lax,11,25
43,Ipoh Coffee,17,25
45,Rogede sild,5,15


## 23. Products that need reordering,
continued
Now we need to incorporate these fields—
UnitsInStock, UnitsOnOrder, ReorderLevel,
Discontinued—into our calculation. We’ll define
“products that need reordering” with the following:
UnitsInStock plus UnitsOnOrder are less than
or equal to ReorderLevel
The Discontinued flag is false (0)

In [57]:
%%sql sqlite:///Northwind2016.sqlite

SELECT CAST(ProductID as int), ProductName, UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued
FROM 'dbo.Products'
WHERE CAST(UnitsInStock as int) + CAST(UnitsOnOrder as int) <= CAST(ReorderLevel as int) 
AND CAST(Discontinued as int) = 0
ORDER BY 1 ASC

Done.


CAST(ProductID as int),ProductName,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
30,Nord-Ost Matjeshering,10,0,15,0
70,Outback Lager,15,10,30,0


## 24. Customer list by region
A salesperson for Northwind is going on a business
trip to visit customers, and would like to see a list of
all customers, sorted by region, alphabetically.
However, he wants the customers with no region
(null in the Region field) to be at the end, instead of
at the top, where you’d normally find the null values.
Within the same region, companies should be sorted
by CustomerID

In [96]:
%%sql sqlite:///Northwind2016.sqlite

SELECT CompanyName, Region, CustomerID, 
CASE WHEN REGION IS NULL THEN 1 ELSE 0 END AS temp_col
FROM 'dbo.Customers'
ORDER BY Region, CustomerID

Done.


CompanyName,Region,CustomerID,temp_col
Alfreds Futterkiste,,ALFKI,0
Ana Trujillo Emparedados y helados,,ANATR,0
Antonio Moreno Taquería,,ANTON,0
Around the Horn,,AROUT,0
Berglunds snabbköp,,BERGS,0
Blauer See Delikatessen,,BLAUS,0
Blondesddsl père et fils,,BLONP,0
Bólido Comidas preparadas,,BOLID,0
Bon app',,BONAP,0
B's Beverages,,BSBEV,0
